<a href="https://colab.research.google.com/github/cowsilver57/sessac_test/blob/main/Chapter5_%EC%9E%91%EC%97%85%ED%98%95_%EC%A0%9C_2%EC%9C%A0%ED%98%95(12%EC%9D%BC%EC%B0%A8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#github와 colab 연동
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata

Cloning into 'yemoonsaBigdata'...
remote: Enumerating objects: 637, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 637 (delta 136), reused 184 (delta 109), pack-reused 363
Receiving objects: 100% (637/637), 10.76 MiB | 14.33 MiB/s, done.
Resolving deltas: 100% (321/321), done.


# 문제

1-1. 주어진 학습 데이터(X_train, y_train)을 활용하여 해당 천체가 은하인지 아닌지 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용

1-2. 얻은 천체의 은하인 경우를 1로 하여 은하의 여부를 다음과 같은 형식의 csv 파일로 생성

(제출한 모델의 성능은 ROC-AUC 평가 지표에 따라 채점)

*   X_test : 천체 관측 특성(평가용)
*   X_train : 천체 관측 특성(학습용)
*   y_train : 은하 여부 데이터(학습용)




# 풀이

##1. 파일 불러오기

In [3]:
import pandas as pd
import numpy as np

X_test = pd.read_csv('/content/yemoonsaBigdata/datasets/Part2/stellar_X_test.csv')
X_train = pd.read_csv('/content/yemoonsaBigdata/datasets/Part2/stellar_X_train.csv')
y_train = pd.read_csv('/content/yemoonsaBigdata/datasets/Part2/stellar_y_train.csv')

print(X_train.info()) #결측치 없음
print(X_train.describe()) #최솟값이 -9999로 이상치 존재

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   alpha     70000 non-null  float64
 1   delta     70000 non-null  float64
 2   u         70000 non-null  float64
 3   g         70000 non-null  float64
 4   r         70000 non-null  float64
 5   i         70000 non-null  float64
 6   z         70000 non-null  float64
 7   redshift  70000 non-null  float64
 8   run_ID    70000 non-null  int64  
 9   rerun_ID  70000 non-null  int64  
 10  cam_col   70000 non-null  int64  
 11  field_ID  70000 non-null  int64  
 12  plate     70000 non-null  int64  
dtypes: float64(8), int64(5)
memory usage: 6.9 MB
None
              alpha         delta             u             g             r  \
count  70000.000000  70000.000000  70000.000000  70000.000000  70000.000000   
mean     177.535809     24.107885     21.938806     20.486718     19.644011   
std       96.4

In [4]:
#이상치 제거
#X_train의 u 칼럼에서 -9999가 아닌 행만 뽑기
y_train = y_train.loc[X_train.u != -9999.000000]
X_train = X_train.loc[X_train.u != -9999.000000]
#수정된 행에 따라 index 새로 고침, drop=True 이전 인덱스는 삭제하고, inplace=True 로 원본 데이터에 저장!
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

print(X_train.describe()) #다시 뽑아봤더니 이상치 없음!
#redshift 열에서 90%~100% 값을 구함.
print(X_train['redshift'].quantile([q/100 for q in range(90,101)]))
#약 2%만이 3을 넘음 (0.99부터 3이 넘는다는 의미!)

              alpha         delta             u             g             r  \
count  69999.000000  69999.000000  69999.000000  69999.000000  69999.000000   
mean     177.535145     24.108238     22.081964     20.629856     19.644032   
std       96.483622     19.664976      2.248554      2.033576      1.852057   
min        0.005528    -18.785328     10.996230     10.498200      9.822070   
25%      127.384192      5.117238     20.350000     18.957435     18.136075   
50%      180.705886     23.580630     22.183900     21.095450     20.122810   
75%      233.942330     39.872079     23.684160     22.121075     21.039035   
max      359.999615     82.947622     32.781390     31.602240     29.571860   

                  i             z      redshift        run_ID  rerun_ID  \
count  69999.000000  69999.000000  69999.000000  69999.000000   69999.0   
mean      19.082984     18.767701      0.575254   4478.702681     301.0   
std        1.756005      1.764734      0.729923   1965.290408  

In [5]:
#파생변수 추가

#'redshift_upper3'라는 새로운 열을 X_train 데이터 프레임에 추가
#'redshift' 값이 3보다 큰 경우, 1 / 아닌 경우 0 부여
X_train['redshift_upper3'] = np.where(X_train['redshift']>3,1,0)
X_test['redshift_upper3'] = np.where(X_test['redshift']>3,1,0)

In [6]:
print(X_train.columns)

Index(['alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'redshift', 'run_ID',
       'rerun_ID', 'cam_col', 'field_ID', 'plate', 'redshift_upper3'],
      dtype='object')


In [7]:
#칼럼 분류
COL_DEL = []
COL_NUM = ['alpha','delta', 'u', 'g', 'r', 'i', 'z','redshift']
COL_CAT = ['run_ID','rerun_ID', 'cam_col', 'field_ID', 'plate', 'redshift_upper3']
COL_Y = ['galaxy']

#X_train 데이터프레임에서 명목형 변수 -> 문자열 형식으로 변환
#훈련 데이터와 테스트 데이터가 동일한 데이터 형식을 가지게 됨.
X_train[COL_CAT] = X_train[COL_CAT].astype(str)
X_test[COL_CAT] = X_test[COL_CAT].astype(str)

#데이터프레임에서 COL_DEL 리스트에 포함된 열들을 제거
X_train = X_train.drop(COL_DEL, axis=1)
X_test = X_test.drop(COL_DEL, axis=1)

In [8]:
#수치형 변수 살펴보기
#X_train 데이터프레임과 y_train 열을 수평으로 연결-> 새로운 데이터프레임 train_df를 생성
train_df = pd.concat([X_train, y_train], axis=1)

#COL_NUM 리스트에 있는 숫자 형식 열의 이름을 하나씩 반복
for _col in COL_NUM:
    print('-'*80)
    print(_col)
    print(train_df.groupby(COL_Y)[_col].describe(), end='\n\n')
#수치형 변수 중에서는 redshift가 가장 의미 있어 보임.
#why? 평균만 봐도 다른 변수들에 비해 차이가 많이 남.

--------------------------------------------------------------------------------
alpha
          count        mean        std       min         25%         50%  \
galaxy                                                                     
0       28433.0  177.234737  99.202036  0.010959  120.997187  180.253146   
1       41566.0  177.740638  94.579769  0.005528  131.392267  181.032298   

               75%         max  
galaxy                          
0       239.025694  359.999615  
1       231.397544  359.989386  

--------------------------------------------------------------------------------
delta
          count       mean        std        min       25%        50%  \
galaxy                                                                  
0       28433.0  25.067356  20.141007 -18.785328  5.060842  26.372836   
1       41566.0  23.452158  19.305416 -12.364701  5.136824  22.255904   

              75%        max  
galaxy                        
0       41.762988  82.947622  
1 

In [9]:
#명목형 변수 살펴보기
for _col in COL_CAT:
    print(train_df.groupby(_col, as_index=False)[COL_Y].mean().sort_values(by=COL_Y, ascending=False), end='\n\n')
#범주형 변수 중 유의미한 변수: cam_col, redshift_upper3
#why? 이 두 변수의 값만 0과 1이 아니고 다른 값을 가짐.

    run_ID  galaxy
97    2883     1.0
325   6121     1.0
399   8054     1.0
38    1897     1.0
245   4800     1.0
..     ...     ...
292   5378     0.0
279   5237     0.0
268   5115     0.0
261   5060     0.0
424    994     0.0

[425 rows x 2 columns]

  rerun_ID    galaxy
0      301  0.593808

  cam_col    galaxy
0       1  0.619151
3       4  0.612120
4       5  0.610490
2       3  0.593031
1       2  0.583152
5       6  0.530405

    field_ID  galaxy
676      717     1.0
750      788     1.0
667      708     1.0
704      746     1.0
706      748     1.0
..       ...     ...
830      934     0.0
829      933     0.0
828      932     0.0
827      931     0.0
835      940     0.0

[847 rows x 2 columns]

     plate  galaxy
6186   999     1.0
4292  6417     1.0
4263  6388     1.0
4264  6389     1.0
1082  1900     1.0
...    ...     ...
1653  2560     0.0
1655  2563     0.0
1656  2564     0.0
3569  5343     0.0
1073  1891     0.0

[6187 rows x 2 columns]

  redshift_upper3    galaxy
0   

In [10]:
#명목형 변수들 인코딩
from sklearn.preprocessing import LabelEncoder
#X_train과 X_test를 연결시킨 새로운 데이터프레임 X를 생성
X = pd.concat([X_train, X_test])
#명목형 변수들을 for문을 활용하여 차례대로 불러옴.
for _col in COL_CAT:
    #LabelEncoder 함수 정의
    le = LabelEncoder()
    #X data를 함수에 맞춤
    le.fit(X[_col])
    #transform 메서드를 호출하면 _col 열의 각 범주값이 해당 범주에 대한 정수로 변환됨.
    X_train[_col] = le.transform(X_train[_col])
    X_test[_col] = le.transform(X_test[_col])

In [11]:
#훈련, 검증 데이터 분할하기
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train)
#stratify 매개변수를 사용하여 클래스 불균형을 고려-> 각 세트가 클래스 비율을 정확하게 반영되어 모델이 편향되거나 일반화되지 않도록 도움

In [12]:
#Standard Scaler 함수를 활용하여 데이터 표준화하기
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_tr[COL_NUM] = scaler.fit_transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.fit_transform(X_val[COL_NUM])
X_test[COL_NUM] = scaler.fit_transform(X_test[COL_NUM])

#X_tr, X_val, y_tr, y_val: train_test_split 함수의 결과로, 훈련,검증 세트를 나타내는 네 개의 데이터 집합을 생성
#X_tr: 훈련 세트의 피처 데이터/ X_val: 검증 세트의 피처 데이터/ y_tr: 훈련 세트의 목표 변수 데이터
#y_val: 검증 세트의 목표 변수 데이터, 검증 세트는 모델의 성능을 평가하고 모델을 튜닝하는데 사용됨.
#모델이 과적합되지 않고 일반화 능력을 가지는지 확인할 수 있음.

In [13]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier()
model_rf.fit(X_tr, y_tr.values.ravel())

RandomForestClassifier()

In [14]:
from xgboost import XGBClassifier
model_xgb1 = XGBClassifier()
model_xgb1.fit(X_tr, y_tr.values.ravel())

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [15]:
from sklearn.metrics import roc_auc_score
y_pred_rf = model_rf.predict_proba(X_val)
y_pred_xgb1 = model_xgb1.predict_proba(X_val)

score_rf = roc_auc_score(y_val, y_pred_rf[:,1])
score_xgb1 = roc_auc_score(y_val, y_pred_xgb1[:,1])

print(score_rf)
print(score_xgb1)

0.9719101035921145
0.8131643353191859


In [16]:
pd.DataFrame({'feature':X_tr.columns, 'fi_rf':model_rf.feature_importances_,'fi_xgb':model_xgb1.feature_importances_})

,feature,fi_rf,fi_xgb
0,alpha,0.016444,0.009727
1,delta,0.015496,0.011454
2,u,0.072099,0.022947
3,g,0.076643,0.090958
4,r,0.045621,0.013915
5,i,0.064344,0.018972
6,z,0.111875,0.040531
7,redshift,0.520376,0.742644
8,run_ID,0.010731,0.010102
9,rerun_ID,0.000000,0.000000


In [17]:
pred = model_xgb1.predict_proba(X_test)[:,1]
pd.DataFrame({'index':X_test.index, 'target':pred}).to_csv('./yemoonsaBigdata/res/201900460.csv', index=False)